6/1/2025

### Model training attempt 1

In [3]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import numpy as np
from sklearn.model_selection import KFold
from tqdm import tqdm

# Dataset class to handle image and mask loading
class KidneySegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, mask_transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.mask_transform = mask_transform
        
        # Collect all case folders (e.g., '00000', '00001', ...)
        self.case_folders = os.listdir(image_dir)
        self.case_folders = [folder for folder in self.case_folders if os.path.isdir(os.path.join(image_dir, folder))]
        
        # Calculate total slices across all cases and store slice counts
        self.slice_counts = []
        for case_id in self.case_folders:
            case_image_dir = os.path.join(self.image_dir, case_id)
            # Count number of images in the case
            num_slices = len([f for f in os.listdir(case_image_dir) if f.endswith('.png')])  # assuming images are .png
            self.slice_counts.append(num_slices)
        
    def __len__(self):
        # The length is the total number of slices in all case folders
        return sum(self.slice_counts)
    
    def __getitem__(self, idx):
        # Determine the case folder and slice number for the given index
        cumulative_slices = 0
        for i, num_slices in enumerate(self.slice_counts):
            cumulative_slices += num_slices
            if idx < cumulative_slices:
                case_id = self.case_folders[i]
                slice_id = idx - (cumulative_slices - num_slices)
                break  # Slice indexing
        
        # Build the paths for the image and mask slice
        img_files = sorted([f for f in os.listdir(os.path.join(self.image_dir, case_id)) if f.endswith('.png')])  # Assuming PNG files
        mask_files = sorted([f for f in os.listdir(os.path.join(self.mask_dir, case_id)) if f.endswith('.png')])  # Assuming PNG files
        
        img_path = os.path.join(self.image_dir, case_id, img_files[slice_id])
        mask_path = os.path.join(self.mask_dir, case_id, mask_files[slice_id])

        # Load the image and mask
        img = Image.open(img_path).convert("RGB")  # Convert to RGB (3 channels)
        mask = Image.open(mask_path).convert("L")  # Convert to grayscale (1 channel)
        
        # Apply any transformations if provided
        if self.transform:
            img = self.transform(img)
        if self.mask_transform:
            mask = self.mask_transform(mask)  # The mask transformation will not change the number of channels
        
        return img, mask

# EfficientNet-B5 Encoder (from torchvision)
class EfficientNetB5Encoder(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNetB5Encoder, self).__init__()
        # Load the pre-trained EfficientNet-B5 model from torchvision
        self.encoder = models.efficientnet_b5(pretrained=pretrained)
        
        # Remove the classification head (fully connected layers) to retain features
        self.encoder = nn.Sequential(*list(self.encoder.children())[:-1])
    
    def forward(self, x):
        # Extract features from the encoder
        features = self.encoder(x)
        return features

# FPN Decoder
class FPNDecoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FPNDecoder, self).__init__()
        # FPN typically uses several layers to combine features from different scales
        self.conv1 = nn.Conv2d(in_channels, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, out_channels, kernel_size=1)
        
        # Instead of fixed upsampling, use adaptive upsampling
        self.upsample = nn.Upsample(size=(512, 512), mode='bilinear', align_corners=True)  # Adaptive resizing to (512, 512)
    
    def forward(self, x):
        # Decode and combine features from multiple scales
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
        # Upsample the output to match the input size (512x512)
        x = self.upsample(x)
        return x

# Final model combining EfficientNet-B5 and FPN
class EfficientNetFPN(nn.Module):
    def __init__(self, out_channels=1, pretrained=True):
        super(EfficientNetFPN, self).__init__()
        # Encoder: EfficientNetB5
        self.encoder = EfficientNetB5Encoder(pretrained=pretrained)
        
        # Decoder: FPN
        self.decoder = FPNDecoder(in_channels=2048, out_channels=out_channels)  # EfficientNet-B5 has 2048 channels in the final layer

    def forward(self, x):
        # Forward pass through encoder and decoder
        features = self.encoder(x)
        out = self.decoder(features)
        return out

# Loss Function: Binary Cross-Entropy (BCE)
def bce_loss(output, target):
    return nn.BCEWithLogitsLoss()(output, target)

# Training loop with 5-fold cross-validation
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm import tqdm

def train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5, checkpoint_dir='checkpoints'):
    # Define transformations
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    mask_transform = transforms.Compose([
        transforms.ToTensor(),
    ])

    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Initialize the dataset
    dataset = KidneySegmentationDataset(image_dir=image_dir, mask_dir=mask_dir, transform=transform, mask_transform=mask_transform)

    # Set up cross-validation
    kfold = KFold(n_splits=num_folds, shuffle=True)

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset), start=1):
        print(f"Training fold {fold}/{num_folds}...")

        # Split dataset
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)

        # Data loaders
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

        # Initialize model, optimizer, and loss function
        model = EfficientNetFPN(out_channels=1, pretrained=True)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Load checkpoint if exists
        checkpoint_path = os.path.join(checkpoint_dir, f"fold_{fold}_checkpoint.pth")
        if os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint['epoch']
            start_batch = checkpoint['batch']
            print(f"Resuming from epoch {start_epoch + 1}, batch {start_batch + 1}")
        else:
            start_epoch = 0
            start_batch = 0
            print("Starting from scratch")
            
            

        # Training loop
        for epoch in range(start_epoch, epochs):
            model.train()
            epoch_loss = 0

            for batch_idx, (images, masks) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"), start=start_batch):
                # Skip already processed batches
                if batch_idx < start_batch:
                    continue

                # Move to device
                images, masks = images.to(device), masks.to(device)

                optimizer.zero_grad()

                # Forward pass
                outputs = model(images)
                loss = bce_loss(outputs, masks)
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()

                # Save progress after each batch
                torch.save({
                    'epoch': epoch,
                    'batch': batch_idx,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                }, checkpoint_path)

            print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader)}")

            # Validation loop
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for images, masks in val_loader:
                    images, masks = images.to(device), masks.to(device)
                    outputs = model(images)
                    loss = bce_loss(outputs, masks)
                    val_loss += loss.item()

            print(f"Validation Loss after Epoch {epoch + 1}: {val_loss / len(val_loader)}")

            # Reset start_batch for the next epoch
            start_batch = 0


# Set directories for your image and mask data
image_dir = "E:/kits23/split_dataset/train/images/"
mask_dir = "E:/kits23/split_dataset/train/masks/"

# Train the model
train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5)


Training fold 1/5...


C:\Users\PRAGNA\AppData\Local\Temp\ipykernel_17392\3045113456.py:171: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


RuntimeError: PytorchStreamReader failed locating file data/8: file not found

### Model training : attempt 2

In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models
from PIL import Image
import numpy as np
from sklearn.model_selection import KFold
from tqdm import tqdm

# Dataset class to handle image and mask loading
class KidneySegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, mask_transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.mask_transform = mask_transform
        
        self.case_folders = [folder for folder in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, folder))]
        self.slice_counts = [
            len([f for f in os.listdir(os.path.join(image_dir, case)) if f.endswith('.png')])
            for case in self.case_folders
        ]
    
    def __len__(self):
        return sum(self.slice_counts)
    
    def __getitem__(self, idx):
        cumulative_slices = 0
        for i, num_slices in enumerate(self.slice_counts):
            cumulative_slices += num_slices
            if idx < cumulative_slices:
                case_id = self.case_folders[i]
                slice_id = idx - (cumulative_slices - num_slices)
                break
        
        img_files = sorted([f for f in os.listdir(os.path.join(self.image_dir, case_id)) if f.endswith('.png')])
        mask_files = sorted([f for f in os.listdir(os.path.join(self.mask_dir, case_id)) if f.endswith('.png')])
        
        img_path = os.path.join(self.image_dir, case_id, img_files[slice_id])
        mask_path = os.path.join(self.mask_dir, case_id, mask_files[slice_id])

        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")
        
        if self.transform:
            img = self.transform(img)
        if self.mask_transform:
            mask = self.mask_transform(mask)
        
        return img, mask

# EfficientNet-B5 Encoder (from torchvision)
class EfficientNetB5Encoder(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNetB5Encoder, self).__init__()
        self.encoder = models.efficientnet_b5(pretrained=pretrained)
        self.encoder = nn.Sequential(*list(self.encoder.children())[:-1])
    
    def forward(self, x):
        return self.encoder(x)

# FPN Decoder
class FPNDecoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FPNDecoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, out_channels, kernel_size=1)
        self.upsample = nn.Upsample(size=(512, 512), mode='bilinear', align_corners=True)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        return self.upsample(x)

# Final model combining EfficientNet-B5 and FPN
class EfficientNetFPN(nn.Module):
    def __init__(self, out_channels=1, pretrained=True):
        super(EfficientNetFPN, self).__init__()
        self.encoder = EfficientNetB5Encoder(pretrained=pretrained)
        self.decoder = FPNDecoder(in_channels=2048, out_channels=out_channels)

    def forward(self, x):
        features = self.encoder(x)
        return self.decoder(features)

# Binary Cross-Entropy Loss
def bce_loss(output, target):
    return nn.BCEWithLogitsLoss()(output, target)

# Safe Save Function
def safe_save(state, filepath):
    temp_path = filepath + ".tmp"
    torch.save(state, temp_path)
    os.rename(temp_path, filepath)

# Training function with checkpointing and cross-validation
def train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5, checkpoint_dir='checkpoints'):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    mask_transform = transforms.Compose([transforms.ToTensor()])
    os.makedirs(checkpoint_dir, exist_ok=True)
    dataset = KidneySegmentationDataset(image_dir, mask_dir, transform=transform, mask_transform=mask_transform)
    kfold = KFold(n_splits=num_folds, shuffle=True)

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset), start=1):
        print(f"Training fold {fold}/{num_folds}...")

        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

        model = EfficientNetFPN(out_channels=1, pretrained=True).to('cuda' if torch.cuda.is_available() else 'cpu')
        optimizer = optim.Adam(model.parameters(), lr=lr)
        checkpoint_path = os.path.join(checkpoint_dir, f"fold_{fold}_checkpoint.pth")
        
        start_epoch, start_batch = 0, 0
        if os.path.exists(checkpoint_path):
            try:
                checkpoint = torch.load(checkpoint_path)
                model.load_state_dict(checkpoint['model_state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
                start_epoch = checkpoint['epoch'] + 1
                start_batch = checkpoint['batch'] + 1
                print(f"Resumed from fold {fold}, epoch {start_epoch}, batch {start_batch}")
            except Exception as e:
                print(f"Failed to load checkpoint: {e}. Starting fold {fold} from scratch.")
                os.remove(checkpoint_path)

        for epoch in range(start_epoch, epochs):
            model.train()
            epoch_loss = 0
            for batch_idx, (images, masks) in enumerate(tqdm(train_loader), start=start_batch):
                if batch_idx < start_batch:
                    continue
                images, masks = images.to('cuda'), masks.to('cuda')
                optimizer.zero_grad()
                outputs = model(images)
                loss = bce_loss(outputs, masks)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                safe_save({
                    'epoch': epoch,
                    'batch': batch_idx,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                }, checkpoint_path)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader)}")
            start_batch = 0

            model.eval()
            val_loss = 0
            with torch.no_grad():
                for images, masks in val_loader:
                    images, masks = images.to('cuda'), masks.to('cuda')
                    outputs = model(images)
                    loss = bce_loss(outputs, masks)
                    val_loss += loss.item()
            print(f"Validation Loss after Epoch {epoch + 1}: {val_loss / len(val_loader)}")

# Set directories and train
image_dir = "E:/kits23/split_dataset/train/images/"
mask_dir = "E:/kits23/split_dataset/train/masks/"
train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5)


Training fold 1/5...


  0%|          | 0/62088 [00:00<?, ?it/s]


AssertionError: Torch not compiled with CUDA enabled

In [7]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import KFold

# Dataset class to handle image and mask loading
class KidneySegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, mask_transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.mask_transform = mask_transform
        
        self.case_folders = [folder for folder in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, folder))]
        self.slice_counts = []
        for case_id in self.case_folders:
            case_image_dir = os.path.join(self.image_dir, case_id)
            num_slices = len([f for f in os.listdir(case_image_dir) if f.endswith('.png')])
            self.slice_counts.append(num_slices)
        
    def __len__(self):
        return sum(self.slice_counts)
    
    def __getitem__(self, idx):
        cumulative_slices = 0
        for i, num_slices in enumerate(self.slice_counts):
            cumulative_slices += num_slices
            if idx < cumulative_slices:
                case_id = self.case_folders[i]
                slice_id = idx - (cumulative_slices - num_slices)
                break
        
        img_files = sorted([f for f in os.listdir(os.path.join(self.image_dir, case_id)) if f.endswith('.png')])
        mask_files = sorted([f for f in os.listdir(os.path.join(self.mask_dir, case_id)) if f.endswith('.png')])
        
        img_path = os.path.join(self.image_dir, case_id, img_files[slice_id])
        mask_path = os.path.join(self.mask_dir, case_id, mask_files[slice_id])
        
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")
        
        if self.transform:
            img = self.transform(img)
        if self.mask_transform:
            mask = self.mask_transform(mask)
        
        return img, mask

# EfficientNet-B5 Encoder
class EfficientNetB5Encoder(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNetB5Encoder, self).__init__()
        self.encoder = models.efficientnet_b5(weights='EfficientNet_B5_Weights.DEFAULT' if pretrained else None)
        self.encoder = nn.Sequential(*list(self.encoder.children())[:-2])
    
    def forward(self, x):
        return self.encoder(x)

# FPN Decoder
class FPNDecoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FPNDecoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, out_channels, kernel_size=1)
        self.upsample = nn.Upsample(size=(512, 512), mode='bilinear', align_corners=True)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.upsample(x)
        return x

# Combined EfficientNet-FPN model
class EfficientNetFPN(nn.Module):
    def __init__(self, out_channels=1, pretrained=True):
        super(EfficientNetFPN, self).__init__()
        self.encoder = EfficientNetB5Encoder(pretrained=pretrained)
        self.decoder = FPNDecoder(in_channels=2048, out_channels=out_channels)

    def forward(self, x):
        features = self.encoder(x)
        return self.decoder(features)

# Loss function
def bce_loss(output, target):
    return nn.BCEWithLogitsLoss()(output, target)

# Training function with checkpointing and cross-validation
def train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5, checkpoint_dir='checkpoints'):
    os.makedirs(checkpoint_dir, exist_ok=True)
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    mask_transform = transforms.ToTensor()
    dataset = KidneySegmentationDataset(image_dir, mask_dir, transform=transform, mask_transform=mask_transform)
    kfold = KFold(n_splits=num_folds, shuffle=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset), start=1):
        print(f"Training fold {fold}/{num_folds}...")
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        model = EfficientNetFPN(out_channels=1, pretrained=True).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        checkpoint_path = os.path.join(checkpoint_dir, f"fold_{fold}_checkpoint.pth")
        start_epoch, start_batch = 0, 0
        
        if os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint['epoch']
            start_batch = checkpoint['batch']
            print(f"Resuming fold {fold} from epoch {start_epoch + 1}, batch {start_batch + 1}")
        else:
            print(f"Starting fold {fold} from scratch.")
        
        for epoch in range(start_epoch, epochs):
            model.train()
            epoch_loss = 0
            for batch_idx, (images, masks) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"), start=start_batch):
                if batch_idx < start_batch:
                    continue
                images, masks = images.to(device), masks.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = bce_loss(outputs, masks)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                torch.save({
                    'epoch': epoch,
                    'batch': batch_idx,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                }, checkpoint_path)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader)}")
            start_batch = 0

            model.eval()
            val_loss = 0
            with torch.no_grad():
                for images, masks in val_loader:
                    images, masks = images.to(device), masks.to(device)
                    outputs = model(images)
                    val_loss += bce_loss(outputs, masks).item()
            print(f"Validation Loss after Epoch {epoch + 1}: {val_loss / len(val_loader)}")

image_dir = "E:/kits23/split_dataset/train/images/"
mask_dir = "E:/kits23/split_dataset/train/masks/"
train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5)


Training fold 1/5...


C:\Users\PRAGNA\AppData\Local\Temp\ipykernel_17392\2391620599.py:121: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=dev

Resuming fold 1 from epoch 1, batch 24


Epoch 1/50:   0%|          | 67/62088 [04:23<67:45:02,  3.93s/it]


KeyboardInterrupt: 

In [10]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import KFold

# Dataset class to handle image and mask loading
class KidneySegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, mask_transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.mask_transform = mask_transform

        self.case_folders = [folder for folder in os.listdir(image_dir) if os.path.isdir(os.path.join(image_dir, folder))]
        self.slice_counts = []
        for case_id in self.case_folders:
            case_image_dir = os.path.join(self.image_dir, case_id)
            num_slices = len([f for f in os.listdir(case_image_dir) if f.endswith('.png')])
            self.slice_counts.append(num_slices)

    def __len__(self):
        return sum(self.slice_counts)

    def __getitem__(self, idx):
        cumulative_slices = 0
        for i, num_slices in enumerate(self.slice_counts):
            cumulative_slices += num_slices
            if idx < cumulative_slices:
                case_id = self.case_folders[i]
                slice_id = idx - (cumulative_slices - num_slices)
                break

        img_files = sorted([f for f in os.listdir(os.path.join(self.image_dir, case_id)) if f.endswith('.png')])
        mask_files = sorted([f for f in os.listdir(os.path.join(self.mask_dir, case_id)) if f.endswith('.png')])

        img_path = os.path.join(self.image_dir, case_id, img_files[slice_id])
        mask_path = os.path.join(self.mask_dir, case_id, mask_files[slice_id])

        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.transform:
            img = self.transform(img)
        if self.mask_transform:
            mask = self.mask_transform(mask)

        return img, mask

# EfficientNet-B5 Encoder
class EfficientNetB5Encoder(nn.Module):
    def __init__(self, pretrained=True):
        super(EfficientNetB5Encoder, self).__init__()
        self.encoder = models.efficientnet_b5(weights='EfficientNet_B5_Weights.DEFAULT' if pretrained else None)
        self.encoder = nn.Sequential(*list(self.encoder.children())[:-2])

    def forward(self, x):
        return self.encoder(x)

# FPN Decoder
class FPNDecoder(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(FPNDecoder, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 256, kernel_size=1)
        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, out_channels, kernel_size=1)
        self.upsample = nn.Upsample(size=(512, 512), mode='bilinear', align_corners=True)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.upsample(x)
        return x

# Combined EfficientNet-FPN model
class EfficientNetFPN(nn.Module):
    def __init__(self, out_channels=1, pretrained=True):
        super(EfficientNetFPN, self).__init__()
        self.encoder = EfficientNetB5Encoder(pretrained=pretrained)
        self.decoder = FPNDecoder(in_channels=2048, out_channels=out_channels)

    def forward(self, x):
        features = self.encoder(x)
        return self.decoder(features)

# Loss function
def bce_loss(output, target):
    return nn.BCEWithLogitsLoss()(output, target)

# Training function with checkpointing and cross-validation
def train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5, checkpoint_dir='checkpoints'):
    os.makedirs(checkpoint_dir, exist_ok=True)
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    mask_transform = transforms.ToTensor()
    dataset = KidneySegmentationDataset(image_dir, mask_dir, transform=transform, mask_transform=mask_transform)
    kfold = KFold(n_splits=num_folds, shuffle=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset), start=1):
        print(f"Training fold {fold}/{num_folds}...")
        train_subset = Subset(dataset, train_idx)
        val_subset = Subset(dataset, val_idx)
        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
        model = EfficientNetFPN(out_channels=1, pretrained=True).to(device)
        optimizer = optim.Adam(model.parameters(), lr=lr)
        checkpoint_path = os.path.join(checkpoint_dir, f"fold_{fold}_checkpoint.pth")
        start_epoch, start_batch = 0, 0

        if os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path, map_location=device)
            model.load_state_dict(checkpoint['model_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint['epoch']
            start_batch = checkpoint['batch']
            print(f"Resuming fold {fold} from epoch {start_epoch + 1}, batch {start_batch + 1}")
        else:
            print(f"Starting fold {fold} from scratch.")

        for epoch in range(start_epoch, epochs):
            model.train()
            epoch_loss = 0
            pbar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", initial=start_batch, total=len(train_loader))
            for batch_idx, (images, masks) in enumerate(pbar, start=start_batch):
                if batch_idx < start_batch:
                    continue
                images, masks = images.to(device), masks.to(device)
                optimizer.zero_grad()
                outputs = model(images)
                loss = bce_loss(outputs, masks)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                torch.save({
                    'epoch': epoch,
                    'batch': batch_idx,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                }, checkpoint_path)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader)}")
            start_batch = 0

            model.eval()
            val_loss = 0
            with torch.no_grad():
                for images, masks in val_loader:
                    images, masks = images.to(device), masks.to(device)
                    outputs = model(images)
                    val_loss += bce_loss(outputs, masks).item()
            print(f"Validation Loss after Epoch {epoch + 1}: {val_loss / len(val_loader)}")

image_dir = "E:/kits23/split_dataset/train/images/"
mask_dir = "E:/kits23/split_dataset/train/masks/"
train_model(image_dir, mask_dir, epochs=50, batch_size=2, lr=0.001, num_folds=5)


Training fold 1/5...


C:\Users\PRAGNA\AppData\Local\Temp\ipykernel_17392\3123974498.py:121: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=dev

Resuming fold 1 from epoch 1, batch 15


Epoch 1/50:   0%|          | 15/62088 [00:07<129:15:10,  7.50s/it]


KeyboardInterrupt: 